In [25]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

import sys
sys.path.append('../task_1')
import autoencoder_classifier as aec

# force reload module
import importlib
importlib.reload(aec)

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger


In [11]:
num_classes = 10
num_epochs = 10

In [20]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


In [21]:
model = aec.AutoencoderClassifier(num_classes=10)

AutoencoderClassifier(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(16, 64, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): ConvTranspose2d(16, 1, kernel_size=(2, 2), stride=(2, 2))
    (3): Sigmoid()
  )
  (classifier): Sequential(
    (0): Linear(in_features=784, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [10]:
model = aec.AutoencoderClassifier(num_classes=10)

# Initialize PyTorch Lightning trainer
trainer = pl.Trainer(gpus=1 if torch.cuda.is_available() else 0, max_epochs=num_epochs, logger=wandb_logger)

# # Start training
# trainer.fit(model, trainloader)



NameError: name 'wandb_logger' is not defined

In [26]:
model = aec.AutoencoderClassifier(num_classes=10)

images = next(iter(trainloader))[0]

out_image, labels = model(images)

print(out_image.shape)


torch.Size([32, 1, 28, 28])


In [28]:
model.encoder(images).shape

torch.Size([32, 16, 7, 7])

In [31]:
from torchsummary import summary

summary(model, (1, 28, 28))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Conv2d-7             [-1, 16, 7, 7]           9,232
              ReLU-8             [-1, 16, 7, 7]               0
   ConvTranspose2d-9             [-1, 64, 7, 7]           9,280
             ReLU-10             [-1, 64, 7, 7]               0
  ConvTranspose2d-11           [-1, 32, 14, 14]          18,464
             ReLU-12           [-1, 32, 14, 14]               0
  ConvTranspose2d-13            [-1, 1, 28, 28]             289
          Sigmoid-14            [-1, 1,